In [1]:
import os
import time
import chipwhisperer as cw
import subprocess
from tqdm.notebook import trange, tnrange
from setup_generic import hardware_setup, scope_reset, tracewhisperer_husky_setup
import numpy as np
import matplotlib.pyplot as plt
from sca_preprocess import trace_fft, trace_butter_lpf
from sca_preprocess import calc_snr, calc_ttest, calc_sod
from sca_preprocess import select_poi_max_rank, select_poi_threshold

TARGET = 'stm32f4'
SS_VER='SS_VER_1_1'

In [16]:
%%bash
make -B TESTFILE=fpr_mul_clip OPT=-O0 build

SS_VER set to SS_VER_1_1
+--------------------------------------------------------
+ target: stm32f4
+ test file: fpr_mul_clip
+ simple serial version: SS_VER_1_1
+ optimization level: -O0
+--------------------------------------------------------
LDSCRIPT build-stm32f4/ldscript.ld
mkdir -p build-stm32f4/obj
   CC    build-stm32f4/obj/fpr_mul_clip.o fpr_mul_clip.c
   CC    build-stm32f4/obj/simpleserial.o /home/kangli/sca/cw-firmware-mcu/simpleserial/simpleserial.c
   CC    build-stm32f4/obj/tracewhisperer.o /home/kangli/sca/cw-firmware-mcu/tracewhisperer/tracewhisperer.c
   CC    build-stm32f4/obj/hal.o /home/kangli/sca/cw-firmware-mcu/hal/hal.c
   CC    build-stm32f4/obj/stm32f4_hal.o /home/kangli/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c


/home/kangli/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c: In function 'platform_init':
/home/kangli/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c:87:9: note: '#pragma message: init by default settings'
   87 | #pragma message "init by default settings"
      |         ^~~~~~~
/home/kangli/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c: In function 'init_uart':
/home/kangli/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c:139:12: note: '#pragma message: Using 38400 baud rate for SS_VER_1_1'
  139 |    #pragma message "Using 38400 baud rate for SS_VER_1_1"
      |            ^~~~~~~


   CC    build-stm32f4/obj/stm32f4_sysmem.o /home/kangli/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_sysmem.c
   CC    build-stm32f4/obj/stm32f4xx_hal.o /home/kangli/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal.c
   CC    build-stm32f4/obj/stm32f4xx_hal_rng.o /home/kangli/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_rng.c
   CC    build-stm32f4/obj/stm32f4xx_hal_gpio.o /home/kangli/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_gpio.c
   CC    build-stm32f4/obj/stm32f4xx_hal_rcc.o /home/kangli/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_rcc.c
   CC    build-stm32f4/obj/stm32f4xx_hal_rcc_ex.o /home/kangli/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_rcc_ex.c
   CC    build-stm32f4/obj/stm32f4xx_hal_uart.o /home/kangli/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_uart.c
   AS    build-stm32f4/obj/stm32f4_startup.o /home/kangli/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_startup.S
   LD    build-stm32f4/fpr_mul_clip.elf build-

In [2]:
TESTFILE='fpr_mul_clip'

In [3]:
# Common setup
PROGRAM = True
scope, target = hardware_setup(
        "build-%s/%s.hex"%(TARGET, TESTFILE), TARGET, SS_VER, PROGRAM)
scope_reset(scope, TARGET)
trace = tracewhisperer_husky_setup(scope, target)

# scope.clock
clkgen_freq0 = scope.clock.clkgen_freq  # 7363636.363636363
scope.clock.clkgen_freq = 30E6
# scope.clock.adc_mul = 4

# scope.gin
scope.gain.db = 12

# target.baud
tmp_target_baud = target.baud
target.baud = int(tmp_target_baud *
                  (scope.clock.clkgen_freq / clkgen_freq0))

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.adc.trig_count                     changed from 1884248                   to 14122674                 
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx                
scope.io.hs2       

In [4]:
def collect_trace_per_class(num, scope, target, trace, N=200):

    text = num.to_bytes(1,byteorder='little')

    # traces = []
    for i in range(N):
        trace.arm_trace()
        scope.arm()
        target.simpleserial_write('p', text)

        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition at', i)
            continue

        # response = target.simpleserial_read('r', 1)
        # print(response)
        # traces.append(scope.get_last_trace())
        traces = scope.get_last_trace()

        # if enable TraceWhisperer
        raw = trace.read_capture_data()
        times = trace.get_rule_match_times(raw, rawtimes=False, verbose=True)

    return traces

In [5]:
def collect_trace_per_class_mul(num, scope, target, trace, start_idx, end_idx, N=200):

    text = num.to_bytes(1,byteorder='little')

    traces = []
    for i in range(N):
        trace.arm_trace()
        scope.arm()
        target.simpleserial_write('p', text)

        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition at', i)
            continue

        # response = target.simpleserial_read('r', 1)
        # print(response)
        tmp = scope.get_last_trace()
        traces.append(tmp[start_idx:end_idx])
        # traces = scope.get_last_trace()

        # if enable TraceWhisperer
        raw = trace.read_capture_data()
        times = trace.get_rule_match_times(raw, rawtimes=False, verbose=True)

    return traces

In [6]:
import itertools
def repeat_set_elements(set_data, N):
    if N < len(set_data):
        raise ValueError("N must be greater than or equal to the number of elements in the set.")
    
    ordered_list = sorted(set_data)
    return list(itertools.islice(itertools.cycle(ordered_list), N))

In [ ]:
#In fpr_mul
#beigin
# w = (uint64_t)x1 * (uint64_t)y0;  
# z1 += (uint32_t)w & 0x01FFFFFF;
# z2 += (uint32_t)(w >> 25);
# zu = (uint64_t)x1 * (uint64_t)y1; 
# z2 += (z1 >> 25);
# z1 &= 0x01FFFFFF;
# zu += z2;
#end

#tracewhisperer:
#start cpu cycles idx: 294
#end cpu cycles idx: 477

start_idx = 280*4
end_idx = 480*4

trace.set_isync_matches(addr0=0x8000c16, addr1=0x8000d48, match='both')
scope.adc.samples = 3500
traces_O0 = []
sets = [
    {4, 5},
    {6, 7},
    {8, 9, 10, 11},
    {12, 13, 14, 15},
    {16, 17, 18, 19, 20, 21, 22},
    {23, 24, 25, 26, 27, 28, 29, 30, 31}
]

for i, s in enumerate(sets):
    for ele in s:
        traces = collect_trace_per_class_mul(ele, scope, target, trace, start_idx, end_idx, N=2000)
        traces_O0.append(traces)
    traces_O0 = np.array(traces_O0)
    print(traces_O0.shape)
    np.save(f'./data/fpr_mul_clip_mul_O0_{i}_2000_each.npy',traces_O0)
    traces_O0=[]

     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294 rule # 0, delta = 294
     477 rule # 0, delta = 183
     294